# Fine-tuning Mistral-7B-Instruct-v0.2 avec Unsloth — Spécialiste SMS Marketing Maroc Telecom

Ce notebook entraîne **Mistral-7B-Instruct-v0.2** avec **Unsloth** (QLoRA) sur le dataset `dataset_fr_10k_ideal.jsonl` (format `instruction`/`input`/`output`) pour produire un **spécialiste SMS marketing Maroc Telecom**.

**Structure (logique Unsloth) :**
1. Installation & Préparation de l'environnement
2. Chargement du modèle & Tokenizer (4-bit) via Unsloth
3. Chargement du dataset JSONL et split Train/Validation
4. Mise en forme (chat template Mistral) + contraintes IAM
5. Configuration LoRA & hyperparamètres
6. Entraînement (SFT)
7. Évaluation rapide (génération) & garde-fous
8. Sauvegarde des adapters, fusion (optionnelle) & export
9. (Optionnel) Publication sur Hugging Face Hub


## 1) Installation & Préparation de l'environnement

In [1]:
!pip -q install unsloth trl transformers accelerate bitsandbytes datasets

import unsloth
import os, json, random, re, torch
from pathlib import Path

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Torch: 2.8.0+cu126
CUDA available: True


In [2]:
import os
# Forcer SDPA (évite les kernels Triton qui explosent sur T4)
os.environ["FLASH_ATTENTION_SKIP_TRITON"] = "1"

# (Optionnel) éviter la fragmentation CUDA
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

# Si transformers >= 4.42
try:
    from transformers import set_default_attn_implementation
    set_default_attn_implementation("sdpa")
except Exception:
    pass


## 2) Chargement du modèle & Tokenizer (Unsloth 4-bit)

In [3]:
import unsloth
from unsloth import FastLanguageModel

MAX_SEQ_LEN = 768
model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = MAX_SEQ_LEN,
    load_in_4bit = True,
    dtype = None,
    device_map = "auto",
)
tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"
print("Loaded:", model_name)


==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Loaded: unsloth/mistral-7b-instruct-v0.2-bnb-4bit


## 3) Chargement du dataset JSONL et split Train/Validation

In [4]:
# (dans la Cellule 3)

from datasets import load_dataset
from pathlib import Path
# Mount Drive + set paths
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH   = "/content/drive/MyDrive/mt_iam/data/dataset_choco_fr_10k_diverse.jsonl"  # <<< tes 10k
OUT_DIR     = "/content/drive/MyDrive/mt_iam/mistral7b_iam_sms_lora"                   # <<< persistant

import os; os.makedirs(os.path.dirname(DATA_PATH), exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)


raw_ds = load_dataset("json", data_files=DATA_PATH, split="train")
raw_ds = raw_ds.shuffle(seed=42)
ds = raw_ds.train_test_split(test_size=0.05, seed=42)
train_ds, val_ds = ds["train"], ds["test"]
print(train_ds.num_rows, val_ds.num_rows)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
9500 500


## 4) Mise en forme — chat template Mistral + règles IAM

In [5]:
# Cellule de formatage (Version stable & rapide)
from datasets import Dataset
import json, re

def _clamp_480(s: str) -> str:
    s = (s or "").strip()
    return s[:480].rstrip() if len(s) > 480 else s

def _build_text(system_prompt: str, user_prompt: str, assistant_text: str) -> str:
    messages = [
        {"role": "system",    "content": system_prompt},
        {"role": "user",      "content": user_prompt},
        {"role": "assistant", "content": assistant_text},
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,   # <<< essentiel pour SFT (inclut la réponse)
    )
    # Assure un EOS en fin de séquence
    if not text.endswith(tokenizer.eos_token):
        text += tokenizer.eos_token
    return text

def format_dataset_batched(examples):
    texts = []
    insts = examples["instruction"]
    inps  = examples["input"]
    outs  = examples["output"]
    for i in range(len(outs)):
        system_prompt = insts[i] or "Tu es un assistant marketing Maroc Telecom."
        user_prompt   = json.dumps(inps[i], ensure_ascii=False)
        assistant     = _clamp_480(outs[i])

        # Filtre soft : évite d'entraîner sur des réponses vides ou trop courtes
        if not assistant or len(assistant) < 40:
            continue

        txt = _build_text(system_prompt, user_prompt, assistant)
        texts.append(txt)
    return {"text": texts}

train_fmt = train_ds.map(
    format_dataset_batched,
    batched=True,
    remove_columns=train_ds.column_names,
    desc="Formatting train",
)
val_fmt = val_ds.map(
    format_dataset_batched,
    batched=True,
    remove_columns=val_ds.column_names,
    desc="Formatting val",
)

print("--- Exemple formaté ---")
print(train_fmt[0]["text"][:400])
print(f"Samples: train={len(train_fmt)} | val={len(val_fmt)}")


--- Exemple formaté ---
<s> [INST] Tu es un rédacteur sms marketing pour Maroc Telecom (IAM). À partir de l’INPUT JSON fourni (profil_client, offer_context, promo_context, cta, deadline, links),  écris UN SEUL SMS promotionnel en français en respectant STRICTEMENT : 1) ≤ 480 caractères ; 2) n’utiliser QUE les chiffres/prix/volumes/durées/destinations présents dans l’input ; 3) Termine TOUJOURS par un appel à l'action cla
Samples: train=9500 | val=500


## 5) Configuration LoRA & Hyperparamètres d'entraînement

In [6]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments , EarlyStoppingCallback
import torch

# Configuration LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    use_rslora = True,
)

# Configuration de l'entraînement optimisée
args = TrainingArguments(
    output_dir = OUT_DIR,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 32,
    num_train_epochs = 1,
    learning_rate = 2e-5,                      # Taux d'apprentissage standard pour LoRA
    logging_steps = 10,
    save_strategy = "steps",
    save_steps = 10,                          # Sauvegarde plus fréquente
    save_total_limit = 3,
    weight_decay = 0.1,
    bf16 = False,
    fp16 = True,
    max_grad_norm = 0.3,
    lr_scheduler_type = "cosine",
    warmup_ratio = 0.1,
    optim = "paged_adamw_8bit",
    report_to = "none",
    eval_strategy = "steps",
    eval_steps = 10,                   # aligné sur save_steps
    load_best_model_at_end = True,
    metric_for_best_model = "eval_loss",
    greater_is_better = False,
)
callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    args = args,
    train_dataset = train_fmt,
    eval_dataset  = val_fmt,
    dataset_text_field = "text",
    packing = True,
    train_on_inputs = False, # Masque le prompt lors du calcul de la loss
    callbacks = callbacks,
)

trainer.model.print_trainable_parameters()

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.8.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758


## 6) Entraînement (SFT)

In [7]:
import os, glob

def get_last_checkpoint(dir_path):
    if not os.path.isdir(dir_path): return None
    ckpts = sorted(glob.glob(os.path.join(dir_path, "checkpoint-*")),
                   key=lambda p: int(p.rsplit("-", 1)[-1]) if p.rsplit("-",1)[-1].isdigit() else -1)
    return ckpts[-1] if ckpts else None

last_ckpt = get_last_checkpoint(OUT_DIR)
print("Dernier checkpoint détecté:", last_ckpt)

if last_ckpt is not None:
    print(f"➡️ Reprise depuis {last_ckpt}")
    train_result = trainer.train(resume_from_checkpoint=last_ckpt)
else:
    print("➡️ Démarrage d'un nouvel entraînement")
    train_result = trainer.train()

trainer.save_model()
train_result.metrics

Dernier checkpoint détecté: /content/drive/MyDrive/mt_iam/mistral7b_iam_sms_lora/checkpoint-290
➡️ Reprise depuis /content/drive/MyDrive/mt_iam/mistral7b_iam_sms_lora/checkpoint-290


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,500 | Num Epochs = 1 | Total steps = 297
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 32
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 32 x 1) = 32
 "-____-"     Trainable parameters = 41,943,040 of 7,283,675,136 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


{'train_runtime': 436.1741,
 'train_samples_per_second': 21.78,
 'train_steps_per_second': 0.681,
 'total_flos': 1.872952618988667e+17,
 'train_loss': 0.002057356004779403}

## 7) Évaluation rapide — génération contrôlée

In [12]:
# --- Étape 8 : Évaluation Qualitative Avancée ---

from unsloth.chat_templates import get_chat_template
import json

# Préparer le tokenizer avec le bon template de chat
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral",
)

# On crée une fonction pour faciliter les tests
def generer_sms(instruction, input_data):
    """Génère un SMS personnalisé à partir d'une instruction et d'un input."""
    # Formatage du prompt de test
    user_prompt = json.dumps(input_data, ensure_ascii=False)
    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": user_prompt},
    ]
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")

    # Génération du message avec des paramètres pour plus de créativité
    outputs = model.generate(input_ids=inputs, max_new_tokens=256, use_cache=True,
                             do_sample=True, top_p=0.9, temperature=0.7)

    resultat_genere = tokenizer.batch_decode(outputs)[0]
    # On nettoie le résultat pour n'afficher que la réponse de l'assistant
    return resultat_genere.split("[/INST]")[-1].strip()


# --- Définition des Cas de Test ---

# L'instruction est la même que pour l'entraînement
instruction_test = "Tu es un rédacteur sms marketing pour Maroc Telecom (IAM). À partir de l’INPUT JSON fourni (profil_client, offer_context, promo_context, cta, deadline, links),  écris UN SEUL SMS promotionnel en français en respectant STRICTEMENT : 1) ≤ 480 caractères ; 2) utiliser UNIQUEMENT les chiffres/prix/volumes/durées/destinations présents dans l’input ; 3) Termine TOUJOURS par un appel à l'action clair (code USSD , lien) ;5) tonalité fluide , naturelle et percutante."

test_cases = [
    {
        "description": "Test 1: CHURN_Competiteur - Offre de rétention agressive",
        "input": {
            "persona": "CHURN_Competiteur",
            "famille": "RISQUE_Churn",
            "cta": "*3",
            "offer_context": {"offre": "Bonus de fidélité", "bonus": "50% de data en plus"}
        }
    },
    {
        "description": "Test 2: OPPORTUNITE_InternetCher - Focus sur les économies",
        "input": {
            "persona": "OPPORTUNITE_InternetCher",
            "famille": "USAGE_Internet",
            "cta": "*3",
            "offer_context": {"offre": "Pass Internet *3", "volume": "2Go", "prix_dh": "20", "economie_potentielle": "90%"}
        }
    },
    {
        "description": "Test 3: PROFIL_AfroTouriste - Offre très spécifique",
        "input": {
            "persona": "PROFIL_AfroTouriste",
            "famille": "USAGE_Roaming",
            "cta": "*77",
            "offer_context": {"offre": "Pass Africa Roaming *77", "destination_exemple": "Sénégal"}
        }
    },
    {
        "description": "Test 4: PROFIL_Econome - Offre à très petit budget",
        "input": {
            "persona": "PROFIL_Econome",
            "famille": "USAGE_Mixte",
            "cta": "*2",
            "offer_context": {"offre": "Pass National *2", "prix_dh": 5, "minutes": "50 min"}
        }
    }
]

# --- Lancement des Tests ---
print("--- LANCEMENT DES TESTS AVANCÉS ---\n")
for case in test_cases:
    print(f"--- {case['description']} ---")
    generated_sms = generer_sms(instruction_test, case['input'])
    print(">>> MESSAGE GÉNÉRÉ :")
    print(generated_sms)
    print("\n" + "="*50 + "\n")

--- LANCEMENT DES TESTS AVANCÉS ---

--- Test 1: CHURN_Competiteur - Offre de rétention agressive ---
>>> MESSAGE GÉNÉRÉ :
IAM vous accueille comme un ami. Profitez jusqu'au 30 août des offres exclusives pour rester fidèle. Bonus de 50% de données en plus pour 1 mois avec *3. Explorez le monde sans limites.</s>


--- Test 2: OPPORTUNITE_InternetCher - Focus sur les économies ---
>>> MESSAGE GÉNÉRÉ :
Maroc Telecom vous offre des connexions sans limites avec le nouveau Pass Internet *3 pour seulement 20 DH • 2Go de données disponibles. Accédez à 90% d'économies sur votre usage internet. Commencez dès aujourd'hui par *3.</s>


--- Test 3: PROFIL_AfroTouriste - Offre très spécifique ---
>>> MESSAGE GÉNÉRÉ :
Maroc Telecom vous invite à découvrir le monde en toute liberté grâce au Pass Africa Roaming *77. Pour 500 DH, gagnez 10 Go de données illimitées pour roamer sur 35 destinations en Afrique. Explorez sans crainte l'Afrique entière. Commencez dès maintenant en appuyant sur *77. Voyagez sa

In [11]:
# --- Patch: générateur SMS robuste (pas de reload, utilise model/tokenizer en mémoire) ---
import json, torch
from transformers import GenerationConfig
from unsloth.chat_templates import get_chat_template

assert "model" in globals(), "❌ `model` absent en mémoire."
assert "tokenizer" in globals(), "❌ `tokenizer` absent en mémoire."

# S'assure du template Mistral
tokenizer = get_chat_template(tokenizer, chat_template="mistral")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

GEN_CFG = GenerationConfig(
    max_new_tokens=180,
    temperature=0.6,
    top_p=0.9,
    repetition_penalty=1.05,
    pad_token_id=tokenizer.eos_token_id,
)

SYSTEM_PROMPT = ("Tu es un rédacteur sms marketing pour Maroc Telecom (IAM). À partir de l’INPUT JSON fourni (profil_client, offer_context, promo_context, cta, deadline, links),  écris UN SEUL SMS promotionnel en français en respectant STRICTEMENT : 1) ≤ 480 caractères ; 2) utiliser UNIQUEMENT les chiffres/prix/volumes/durées/destinations présents dans l’input ; 3) Termine TOUJOURS par un appel à l'action clair (code USSD , lien) ;5) tonalité fluide , naturelle et percutante."

)

def generer_sms(sample: dict, gen_cfg: GenerationConfig = GEN_CFG) -> str:
    """Construit le prompt (system+user JSON) et génère 1 SMS en ≤480 caractères."""
    model.eval()
    device = next(model.parameters()).device

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": json.dumps(sample, ensure_ascii=False)},
    ]

    # 1) Obtenir du TEXTE via le template (pas de tensor ici)
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    # 2) Re-tokenizer en DICT -> input_ids + attention_mask
    enc = tokenizer(
        prompt_text,
        return_tensors="pt",
        add_special_tokens=False,
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    input_len = enc["input_ids"].shape[-1]

    with torch.no_grad():
        out = model.generate(
            input_ids=enc["input_ids"],
            attention_mask=enc.get("attention_mask", torch.ones_like(enc["input_ids"])),
            max_new_tokens=gen_cfg.max_new_tokens,
            do_sample=True,
            temperature=gen_cfg.temperature,
            top_p=gen_cfg.top_p,
            repetition_penalty=gen_cfg.repetition_penalty,
            pad_token_id=gen_cfg.pad_token_id,
        )

    gen_tokens = out[0, input_len:]  # uniquement la partie générée
    sms = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
    return sms[:480]

# 5) Trois tests rapides
tests = [
    # Internet / *3
    {
        "persona":"PROFIL_Internet","famille":"USAGE_Internet","cta":"*3",
        "offer_context":{"volume":"5 Go","validite":"30 jours","prix_dh":50},
        "links":{"details":"https://offres.iam.ma"}
    },
    # Roaming multiservices / *7
    {
        "persona":"PROFIL_VoyageurComplet","famille":"USAGE_Roaming","cta":"*7",
        "offer_context":{"data":"6 Go","voix":"2 H","sms":"200","prix_dh":200,"validite":"30 jours","destinations":"111"},
        "deadline":"31/12"
    },
    # Smartphone e-boutique
    {
        "persona":"OPPORTUNITE_AchatNouveaute","famille":"OPPORTUNITE_Achat_Equipement","cta":"e-boutique",
        "offer_context":{"modele":"Samsung Galaxy A16 128Go","prix_dh":"2249","avantage":"livraison gratuite"},
        "links":{"achat":"https://offres.iam.ma/smartphones"}
    },
]

for i, sample in enumerate(tests, 1):
    sms = generer_sms(sample)
    print(f"\n=== TEST {i} ===")
    print(sms)


=== TEST 1 ===
Explorez le monde virtuel sans limites avec IAM. Profitez des 5 Go illimites pendant 30 jours à seulement 50 DH. Inscrivez-vous aujourd'hui via *3 pour accéder à cette opportunité exclusive. Commencez votre aventure numérique dès maintenant.

=== TEST 2 ===
Restez connectés avec IAM. Offrez jusqu'à 200 DH pour des données illimitées pendant 30 jours dans 111 destinations. Commencez dès aujourd'hui via *7. Expérimentez le voyage sans limites.

=== TEST 3 ===
Explorez des smartphones innovants chez Maroc Telecom. Offrez aujourd'hui le Samsung Galaxy A16 128Go à seulement 2249 DH, livrés gratuitement. Commencez votre aventure numérique dès maintenant sur notre e-boutique.
